<a href="https://colab.research.google.com/github/LightGreenWorld/INCEptTION_Annotation/blob/main/Cohere_RAG_COVID_Regulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -qU cohere langchain langchain-cohere langchain_community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


### Cohere Embedding model - Retriever

#### Retriever에 사용할 임베딩 모델
 Cohere Embed Models: https://docs.cohere.com/cohere-embed

In [25]:
!pip install python-dotenv

In [35]:
from os import path
import os

if os.path.exists("Env/.env.txt"):
  try:
      print("파일이 성공적으로 변경됐습니다.")
      os.rename("Env/.env.txt", "Env/.env")
  except FileNotFoundError:
      print("파일이 존재하지 않습니다.")
      exit(1)
  except OSError as e:
      print(f"파일 이름 변경 중 오류 발생: {e}")
      exit(1)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("Env/.env")

cohere_api_key = os.getenv("Cohere_API_KEY")
print(cohere_api_key)

### 0. TEST with English Sample

In [37]:
from langchain_cohere import CohereEmbeddings

cohere_embeddings = CohereEmbeddings(model="embed-english-light-v3.0")
text = "This is a test document."

embedding_result = cohere_embeddings.embed_query(text)
print(embedding_result)

[-0.09338379, 0.0871582, -0.03326416, 0.01953125, 0.07702637, 0.034729004, -0.058380127, -0.031021118, -0.030517578, -0.055999756, 0.050842285, -0.006752014, 0.038391113, -0.0014362335, -0.041137695, -0.008880615, 0.026000977, -0.023010254, 0.05456543, -0.03366089, 0.055633545, 0.028579712, -0.068603516, 0.03970337, -0.06677246, 0.06732178, -0.013053894, -0.0060920715, 0.038116455, 0.057800293, 0.048736572, 0.026855469, 0.009849548, 0.08312988, 0.073791504, 0.01663208, -0.0871582, 0.01802063, -0.0020828247, -0.0031356812, 0.039978027, -0.03164673, 0.009796143, 0.011375427, 0.0068855286, 0.092285156, 0.05218506, -0.060943604, 0.038269043, -0.018218994, -0.04510498, -0.0847168, 0.008300781, -0.060058594, 0.0012111664, 0.05102539, 0.05218506, -0.047210693, -0.051239014, -0.044158936, -0.058166504, 0.07849121, -0.019165039, 0.06451416, 0.024887085, 0.011405945, -0.03768921, -0.018814087, -0.06829834, -0.052825928, -0.019104004, -0.021194458, 0.043518066, 0.07525635, 0.082336426, 0.00376510

In [7]:
print("Embedding Dim :", len(embedding_result))

Embedding Dim : 384


### 1. pdf 텍스트 Load

> Sample pdf(Kor): https://www.work24.go.kr/cm/main.do?bbs_seq=20210601899


In [8]:
!pip install -U pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.7 MB/s eta 0:00:00


In [9]:
# Trial 1 to load PDF
from langchain_core.documents.base import Blob
blob = Blob.from_path("/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/Docs/COVID19_Regulation_KO.pdf")
print(blob)

metadata={} mimetype='application/pdf' path='/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf'


In [10]:
# Trial 2 to load PDF
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/Docs/RAG_Langchain/COVID19_Regulation_KO.pdf")
documents = loader.load()

documents

# for doc in documents:
#   print(doc.page_content)

[Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='□코로나19관련 안전보건교육 지침 개정 안내(교육지침 4판)-안전·보건사고 발생위험 상승 우려로 인해 기존 유예되었던 교육을 인력기준 및 방역수칙 준수하여 정상 실시구분현 행(3판) 개 정(4판) 비고(사유 등)직무교육【이수기간 유예】ㅇ직무교육 대상자가 교육\xa0 수강을 원치 않을 경우-「감염병 재난」 상황\xa0 \xa0         해제날부터 6개월 이내에\xa0 직무교육을 이수 \n【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만\n 안전보건사고 발생 위험상승 우려 근로자정기교육\n【소규모 교육】◦ 정기교육 시 전직원 집체\xa0 교육 보다는- 가급적 ‘소규모 단위’\xa0 교육을 권장◦ 매일 또는 격일 단위로\xa0 분할 실시하되, - 가급적 1회 30분을 넘지\xa0 않도록 할 것\n【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만◦ 안전보건교육으로 볼 수\xa0 없는 직무교육이 혼재되는\xa0 부분은\xa0- 명확히 구분하여 실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷\xa0 원격교육) 시- 관리감독자 집체 또는\xa0 현장교육 유예【교

In [11]:
documents_wo_stop = documents[0].page_content.replace('\xa0', '')
documents[0].page_content = documents_wo_stop
documents

[Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='□코로나19관련 안전보건교육 지침 개정 안내(교육지침 4판)-안전·보건사고 발생위험 상승 우려로 인해 기존 유예되었던 교육을 인력기준 및 방역수칙 준수하여 정상 실시구분현 행(3판) 개 정(4판) 비고(사유 등)직무교육【이수기간 유예】ㅇ직무교육 대상자가 교육 수강을 원치 않을 경우-「감염병 재난」 상황          해제날부터 6개월 이내에 직무교육을 이수 \n【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만\n 안전보건사고 발생 위험상승 우려 근로자정기교육\n【소규모 교육】◦ 정기교육 시 전직원 집체 교육 보다는- 가급적 ‘소규모 단위’ 교육을 권장◦ 매일 또는 격일 단위로 분할 실시하되, - 가급적 1회 30분을 넘지 않도록 할 것\n【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여 실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷 원격교육) 시- 관리감독자 집체 또는 현장교육 유예【교육실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만

#### 1-2. 텍스트 split 하기

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [13]:
texts

[Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='□코로나19관련 안전보건교육 지침 개정 안내(교육지침 4판)-안전·보건사고 발생위험 상승 우려로 인해 기존 유예되었던 교육을 인력기준 및 방역수칙 준수하여 정상 실시구분현'),
 Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='상승 우려로 인해 기존 유예되었던 교육을 인력기준 및 방역수칙 준수하여 정상 실시구분현 행(3판) 개 정(4판) 비고(사유 등)직무교육【이수기간 유예】ㅇ직무교육 대상자가 교육'),
 Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'c

#### 1-3. FAISS(Vector Store) 적재하기

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

retriever = FAISS.from_documents(texts,
            CohereEmbeddings(model="embed-multilingual-v3.0")).as_retriever(search_kwargs={"k":10})

#### 1-4. Retriever 생성 및 문서 검색



In [16]:
def pretty_print_docs(docs):
  print(
      f"\n{'-'*100}\n".join(
          [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
      )
  )

In [17]:
query = "2.5단계 이상에서 관리 감독자 교육 시 몇명까지 수용 가능한가요?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여 실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷 원격교육) 시-
----------------------------------------------------------------------------------------------------
Document 2:

【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여
----------------------------------------------------------------------------------------------------
Document 3:

실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷 원격교육) 시- 관리감독자 집체 또는 현장교육 유예【교육실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지
----------------------------------------------------------------------------------------------------
Document 4:

집체 또는 현장교육 유예【교육실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만
----------------------------------------------------------------------------------------------------
Document 5:

【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만
-----------------------------------------------------------------------------------------------

### Cohere Rerank model - Rerandker

#### ContextualCompressionRetriever 활용하여 Reranker 추가

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=5)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=retriever
                                                       )
compressed_docs = compression_retriever.invoke(
    "2.5단계 이상에서 관리 감독자 교육 시 몇명까지 수용 가능한가요?"
)
pretty_print_docs(compressed_docs)


Document 1:

【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만
----------------------------------------------------------------------------------------------------
Document 2:

【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여
----------------------------------------------------------------------------------------------------
Document 3:

집체 또는 현장교육 유예【교육실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만
----------------------------------------------------------------------------------------------------
Document 4:

50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여 실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷 원격교육) 시-
----------------------------------------------------------------------------------------------------
Document 5:

실시 관리감독자교육【집체·현장 유예】◦ 관리감독자 정기교육(인터넷 원격교육) 시- 관리감독자 집체 또는 현장교육 유예【교육실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지


### RAG

>[Langchain Docs : create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)

>[Langchain Docs : create_retrieval_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html)

In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere

llm = ChatCohere()

system_prompt = (
    "Use the given context to answer the question in Korean. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(compression_retriever, combine_docs_chain)

In [20]:
query = "2.5단계 이상에서 관리 감독자 교육 시 몇명까지 수용 가능한가요?"
response = chain.invoke({"input": query})

In [21]:
response

{'input': '2.5단계 이상에서 관리 감독자 교육 시 몇명까지 수용 가능한가요?',
 'context': [Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'relevance_score': 0.6978149}, page_content='【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만'),
  Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'relevance_score': 0.6974029}, page_content='【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 

In [22]:
response['input']

'2.5단계 이상에서 관리 감독자 교육 시 몇명까지 수용 가능한가요?'

In [23]:
response['context']

[Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'relevance_score': 0.6978149}, page_content='【이수기간 유예 종료】◦ 직무교육 이수기간 유예종료 - 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만'),
 Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.10141', 'creationdate': '2021-06-21T09:41:20+09:00', 'author': 'Moel', 'moddate': '2021-06-21T09:41:20+09:00', 'pdfversion': '1.4', 'source': '/content/drive/MyDrive/Colab Notebooks/RAG_Langchain/COVID19_Regulation_KO.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'relevance_score': 0.6974029}, page_content='【교육 실시】◦ 방역지침을 준수하여 교육 실시* 2단계까지 100미만, 2.5단계 이상 50인 미만◦ 안전보건교육으로 볼 수 없는 직무교육이 혼재되는 부분은- 명확히 구분하여'),
 Document(metada

In [24]:
print(response['answer'])

2.5단계 이상에서 관리 감독자 교육 시 **50인 미만**까지 수용 가능합니다. 이는 제공된 맥락에서 다음과 같은 내용을 통해 확인할 수 있습니다:

- **방역지침을 준수하여 교육 실시**: 2.5단계 이상에서는 50인 미만으로 교육이 실시됩니다.
- **관리감독자교육**: 집체 또는 현장교육이 유예되며, 인터넷 원격교육으로 진행됩니다.

따라서, 2.5단계 이상에서 관리 감독자 교육은 50인 미만으로 수용 가능합니다.
